In [97]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Load Data

In [99]:
path = "input/Online Retail.xlsx"
df = pd.read_excel(path)
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


# Preprocess Data

In [101]:
def preprocess_df(df):
    return (df
        .dropna(subset="CustomerID")
        .dropna(subset="InvoiceDate")
        .assign(
            customer_id = lambda df_: df_["CustomerID"].astype(int), 
            transaction_date = lambda df_: pd.to_datetime(df_["InvoiceDate"]), 
        )
        .loc[:, ["transaction_date", "customer_id"]]
        .dropna(how="any")
        .reset_index(drop=True)
    )

df_clean = preprocess_df(df)

# Cohort Analysis

In [271]:
def fillna_diagonal_lower_right(df):

    df = df.astype(float)

    # # set the diagonal elements to NaN
    # np.fill_diagonal(np.fliplr(df.values), np.nan)
    # get the lower right quadrant
    rows, cols = np.tril_indices(len(df), k=-1)
    reversed_cols = len(df) - 1 - cols  
    df.values[rows, reversed_cols] = np.nan
    return df

def make_cohort(df: pd.DataFrame) -> pd.DataFrame:
    """Do a cohort analysis.

    Args:
        df (pd.DataFrame): A DF with two columns; ["transaction_date", "customer_id"].

    Returns:
        pd.DataFrame: A pivoted DF with cohort.
    """

    df_result = (df
        .sort_values(["transaction_date", "customer_id"])
        .assign(
            # customer first purchase
            first_purchase = lambda df_: (df_
                                        .groupby(["customer_id"])["transaction_date"]
                                        .transform("min") + 
                                        pd.offsets.MonthEnd(0) - 
                                        pd.offsets.MonthBegin(1)
                                        ).dt.date,
            # distance betweeen first purchase and transaction date (in month)
            months_after_first_transaction = lambda df_: (
                # convert transaction date to first day of the month
                (df_["transaction_date"] + pd.offsets.MonthEnd(0) - pd.offsets.MonthBegin(1)).dt.to_period("M")
                - pd.to_datetime(df_["first_purchase"]).dt.to_period("M")
            ).apply(lambda x: x.n) # cast to int 
        )
        .groupby(["first_purchase", "months_after_first_transaction"])
        .agg(
            num_cust = ("customer_id", "nunique")
        )
        # get the initial number of customer/customer in month 0 for denominator
        .assign(
            num_cust_first_month = lambda df_: np.where(
                df_.index.get_level_values("months_after_first_transaction") == 0, 
                df_["num_cust"], np.nan
            )
        )
        .assign(
            # ffill to get the denominator
            num_cust_first_month = lambda df_: df_["num_cust_first_month"].ffill(),
            # get the percentage of each month relative to month 0
            percentage_to_num_cust_first_month = lambda df_: df_["num_cust"] / df_["num_cust_first_month"]
        )
        # pivot
        .reset_index()
        .pivot(
            index = "first_purchase",
            columns = "months_after_first_transaction", 
            values = "percentage_to_num_cust_first_month"
        )
        # fillna in case if there is month without buyer
        # however, this will fill the lower right diagonal with 0
        .fillna(0)
        # fill diagonal with na again
        .pipe(fillna_diagonal_lower_right)
        # rename axis
        .rename_axis("Months after First Transaction", axis=1)
        .rename_axis("First Transaction", axis=0)
    )
    # format the index
    df_result.index = pd.to_datetime(df_result.index).strftime('%b %Y')

    # get average per months after transaction
    df_result = (df_result
        .transpose()
        .assign(Average=lambda df_: df_.mean(axis=1))
        .transpose()
    )
    return df_result

In [272]:
df_cohort = make_cohort(df_clean)
df_cohort

Months after First Transaction,0,1,2,3,4,5,6,7,8,9,10,11,12
First Transaction,,,,,,,,,,,,,
Dec 2010,1.0,0.381857,0.334388,0.387131,0.359705,0.396624,0.379747,0.354430,0.354430,0.394515,0.373418,0.500000,0.274262
Jan 2011,1.0,0.239905,0.282660,0.242280,0.327791,0.299287,0.261283,0.256532,0.311164,0.346793,0.368171,0.149644,NaN
Feb 2011,1.0,0.247368,0.192105,0.278947,0.268421,0.247368,0.255263,0.281579,0.257895,0.313158,0.092105,NaN,NaN
Mar 2011,1.0,0.190909,0.254545,0.218182,0.231818,0.177273,0.263636,0.238636,0.288636,0.088636,NaN,NaN,NaN
Apr 2011,1.0,0.227425,0.220736,0.210702,0.207358,0.237458,0.230769,0.260870,0.083612,NaN,NaN,NaN,NaN
May 2011,1.0,0.236559,0.172043,0.172043,0.215054,0.243728,0.265233,0.103943,NaN,NaN,NaN,NaN,NaN
Jun 2011,1.0,0.208511,0.187234,0.272340,0.246809,0.336170,0.102128,NaN,NaN,NaN,NaN,NaN,NaN
Jul 2011,1.0,0.209424,0.204188,0.230366,0.272251,0.115183,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Aug 2011,1.0,0.251497,0.251497,0.251497,0.137725,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Check Result

In [291]:
first_month = df_clean.loc[
    (df_clean["transaction_date"].dt.month == 12) & (df_clean["transaction_date"].dt.year == 2010), 
    "customer_id"
].unique()

second_month = df_clean.loc[
    (df_clean["transaction_date"].dt.month == 1) & (df_clean["transaction_date"].dt.year == 2011) &
    (df_clean["customer_id"].isin(first_month)), 
    "customer_id"
].unique()

third_month = df_clean.loc[
    (df_clean["transaction_date"].dt.month == 2) & (df_clean["transaction_date"].dt.year == 2011) &
    (df_clean["customer_id"].isin(first_month)), 
    "customer_id"
].unique()

fourth_month = df_clean.loc[
    (df_clean["transaction_date"].dt.month == 3) & (df_clean["transaction_date"].dt.year == 2011) &
    (df_clean["customer_id"].isin(first_month)), 
    "customer_id"
].unique()

print(len(second_month) / len(first_month))
print(len(third_month) / len(first_month))
print(len(fourth_month) / len(first_month))

0.3818565400843882
0.33438818565400846
0.3871308016877637
